# River input data
**Author: Jun Sasaki  Coded on 2025-05-10 Updated on 2025-09-03**<br>
`river_input_checker.ipynb` should be used for checking river data, where the mesh is created from the grd file. This notebook has similar functionality, with the primary difference being the creation of a mesh from the FVCOM output netCDF file.

In [ ]:
from pathlib import Path
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles
import matplotlib.pyplot as plt

from xfvcom import FvcomDataLoader
from xfvcom import FvcomPlotter, FvcomPlotConfig, FvcomPlotOptions
from xfvcom import parse_river_namelist
from xfvcom import make_node_marker_post

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

png_dir = Path("PNG"); png_dir.mkdir(exist_ok=True)

## Loading FVCOM output and input files
It will take long time (several minutes).

In [ ]:
base_path = Path("~/Github/TB-FVCOM/goto2023").expanduser()
ncfile = "tb_w1.8_r16_2020_0001.nc"
obcfile = "TokyoBay18_obc.dat"
# nc_river = "TokyoBayfinal_river.nc"
river_nc = "TokyoBayfinal16_river.nc"
river_nml = "RIVERS_NAMELIST16.nml" 
ncfile_path = base_path / "output/2020" / ncfile
obcfile_path = base_path / "input/2020" / obcfile
river_nc_path = base_path / "input/2020" / river_nc
river_nml_path = base_path / "input/2020" / river_nml
river = FvcomDataLoader(ncfile_path=river_nc_path, time_tolerance=5)
df = parse_river_namelist(river_nml_path)

fvcom = FvcomDataLoader(ncfile_path=ncfile_path, obcfile_path=obcfile_path, time_tolerance=5)
cfg = FvcomPlotConfig()
plotter = FvcomPlotter(fvcom.ds, cfg)

## Plot river discharge nodes

In [ ]:
mkw = {"marker": "o", "color": "red", "markersize": 3, "zorder": 4}
tkw = {"fontsize": 8, "color": "yellow", "ha": "center", "va": "bottom",
       "zorder": 5, "clip_on": True}

# Convert 0-based indices to 1-based for make_node_marker_post
# (df.grid_location is 0-based from parse_river_namelist with default to_zero_based=True)
river_nodes_1based = df.grid_location + 1

pp = make_node_marker_post(
    river_nodes_1based,  # Pass 1-based indices
    plotter,
    marker_kwargs=mkw,
    text_kwargs=tkw,
    index_base=1,
)

opts = FvcomPlotOptions(
    figsize=(6,9),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="#ffffff",
    mesh_linewidth=0.3,
    xlim = ("139:40:00", "140:00:00"),
    ylim = ("35:12:00", "35:42"),
    lon_tick_skip=2,
)

plotter.plot_2d(da=None, post_process_func=pp, opts=opts)